<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Phidata/phi_groq_content_analysis_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Research Assistant: Automated Content Analysis and Summarization

## Introduction
This notebook implements an intelligent research assistant using the Phi framework and Groq's Mixtral-8x7B model. The assistant is designed to automatically gather, analyze, and summarize information on any given topic by leveraging web search capabilities and natural language processing.

## Goals
1. **Automated Research**: Enable automated collection of relevant information from multiple web sources on any specified topic.
2. **Content Analysis**: Process and analyze web content to extract key insights and relevant information.
3. **Smart Summarization**: Generate concise, well-structured summaries with key bullet points from the gathered information.
4. **Real-time Updates**: Provide up-to-date information by actively searching current web sources.

## Key Components
- **Groq Integration**: Utilizing Groq's Mixtral-8x7B model for advanced language processing
- **DuckDuckGo Search**: Web search functionality for gathering relevant sources
- **Newspaper4k**: Web scraping and article extraction capabilities
- **SQLite Storage**: Persistent storage for maintaining conversation history and session data

This notebook demonstrates how to build an AI-powered research assistant that can help users quickly gather and synthesize information on any topic of interest, saving valuable research time while providing comprehensive insights.

In [8]:
%pip install -qU phidata yfinance duckduckgo-search newspaper4k lxml_html_clean groq

In [11]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [9]:
# Import libraries
from phi.agent import Agent
from phi.model.groq import Groq
from phi.storage.agent.sqlite import SqlAgentStorage
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.newspaper4k import Newspaper4k
from phi.run.response import RunEvent, RunResponse

In [16]:
# Create directory for SQLite database
os.makedirs("tmp", exist_ok=True)

# Define the agent
agent = Agent(
    # Using a valid Groq model ID
    model = Groq(id="mixtral-8x7b-32768"),
    tools=[DuckDuckGo(), Newspaper4k()],
    description="You are a researcher writing an article on a topic.",
    instructions=[
        "For a given topic, search for the top 5 links.",
        "Then read each URL and extract the article text.",
        "Analyse and prepare 3-5 bullet points based on the information.",
    ],
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
    add_history_to_messages=True,
    storage=SqlAgentStorage(table_name="agent_sessions", db_file="tmp/agent.db"),
)

# Function to convert response to a stream
def as_stream(response):
    for chunk in response:
        if isinstance(chunk, RunResponse) and isinstance(chunk.content, str):
            if chunk.event == RunEvent.run_response:
                yield chunk.content

# Test the agent
prompt = "Tell me about the latest advancements in AI."
print(f"User: {prompt}")

try:
    print("Assistant:")
    chunks = agent.run(prompt, stream=True)
    for chunk in as_stream(chunks):
        print(chunk, end="")
except Exception as e:
    print(f"Error occurred: {str(e)}")

User: Tell me about the latest advancements in AI.
Assistant:

Running:
 - duckduckgo_news(query=latest advancements in AI 2025)

Here are the latest advancements in AI as of 2025-01-18:

- **The Rise Of AI Leaders: Meet The Superhumans** ([Forbes](https://www.forbes.com/councils/forbestechcouncil/2025/01/16/the-rise-of-ai-leaders-meet-the-superhumans/))
  - AI is not just about automation or convenience, but amplifying human capabilities and solving complex global problems.

- **Quantum Computing Could Achieve Singularity In 2025\-\-A ChatGPT Moment** ([Forbes](https://www.forbes.com/sites/luisromero/2025/01/18/quantum-computing-could-achieve-singularity-in-2025-a-chatgpt-moment/))
  - Quantum computing is getting more attention, sparking questions about what QC could achieve in 2025.

- **AI on AI: Popular Large Language Models Weigh In on What's Next for AI in 2025** ([cc.gatech.edu](https://www.cc.gatech.edu/news/ai-ai-popular-large-language-models-weigh-whats-next-ai-2025))
  - La